In [1]:
import pandas as pd

df = pd.read_csv('ecommerce_sales_data.csv')

In [2]:
df.head()

,Order Date,Product Name,Category,Region,Quantity,Sales,Profit
0,2024-12-31,Printer,Office,North,4,3640,348.93
1,2022-11-27,Mouse,Accessories,East,7,1197,106.53
2,2022-05-11,Tablet,Electronics,South,5,5865,502.73
3,2024-03-16,Mouse,Accessories,South,2,786,202.87
4,2022-09-10,Mouse,Accessories,West,1,509,103.28


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order Date    3500 non-null   object 
 1   Product Name  3500 non-null   object 
 2   Category      3500 non-null   object 
 3   Region        3500 non-null   object 
 4   Quantity      3500 non-null   int64  
 5   Sales         3500 non-null   int64  
 6   Profit        3500 non-null   float64
dtypes: float64(1), int64(2), object(4)
memory usage: 191.5+ KB


In [5]:
df.describe(include='all')

,Order Date,Product Name,Category,Region,Quantity,Sales,Profit
count,3500,3500,3500,3500,3500.000000,3500.000000,3500.000000
unique,1051,10,3,4,NaN,NaN,NaN
top,2023-08-12,Monitor,Electronics,West,NaN,NaN,NaN
freq,11,364,1742,898,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,4.931714,3047.966000,527.047203
std,NaN,NaN,NaN,NaN,2.575895,2440.213237,504.139732
min,NaN,NaN,NaN,NaN,1.000000,51.000000,6.970000
25%,NaN,NaN,NaN,NaN,3.000000,1049.500000,158.695000
50%,NaN,NaN,NaN,NaN,5.000000,2350.500000,361.070000
75%,NaN,NaN,NaN,NaN,7.000000,4537.000000,729.125000


In [6]:
df.isnull().sum()

Order Date      0
Product Name    0
Category        0
Region          0
Quantity        0
Sales           0
Profit          0
dtype: int64

In [7]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

In [8]:
df.columns

Index(['order_date', 'product_name', 'category', 'region', 'quantity', 'sales',
       'profit'],
      dtype='object')

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'],errors = 'coerce')
df['quantity'] = pd.to_numeric(df['quantity'],errors = 'coerce').astype('Int64')
df['sales'] = pd.to_numeric(df['sales'],errors = 'coerce')
df['profit'] = pd.to_numeric(df['profit'],errors = 'coerce')

In [10]:
## create of columns
# margin_rate
df['margin_rate'] = df['profit']/df['sales']
# average_price_per_unit
df['average_price_per_unit'] = df['sales']/df['quantity']
# year_month
df['year_month'] = df['order_date'].dt.to_period('M').astype(str)
# sales_performance_level
df['sales_performance_level'] = pd.qcut(df['sales'], q=3,labels = ['low', 'medium', 'high'])
# profitability_level
df['profitability_level'] = pd.qcut(df['margin_rate'],q=3,labels=['low_margin', 'moderate_margin', 'high_margin'])
# contribution_to_total_profit
total_profit = df['profit'].sum()
df['contribution_to_total_profit (%)'] = (df['profit'] / total_profit) * 100
# convert_margin_rate
df['margin_rate (%)'] = df['margin_rate'] * 100

In [11]:
df.head()

,order_date,product_name,category,region,quantity,sales,profit,margin_rate,average_price_per_unit,year_month,sales_performance_level,profitability_level,contribution_to_total_profit (%),margin_rate (%)
0,2024-12-31,Printer,Office,North,4,3640,348.93,0.095860,910.0,2024-12,medium,low_margin,0.018916,9.585989
1,2022-11-27,Mouse,Accessories,East,7,1197,106.53,0.088997,171.0,2022-11,low,low_margin,0.005775,8.899749
2,2022-05-11,Tablet,Electronics,South,5,5865,502.73,0.085717,1173.0,2022-05,high,low_margin,0.027253,8.571697
3,2024-03-16,Mouse,Accessories,South,2,786,202.87,0.258104,393.0,2024-03,low,high_margin,0.010998,25.810433
4,2022-09-10,Mouse,Accessories,West,1,509,103.28,0.202908,509.0,2022-09,low,moderate_margin,0.005599,20.290766


In [12]:
df.to_csv("e_commerce_sales_data_clean.csv",index=False,encoding="utf-8")

In [13]:
df.to_excel("e_commerce_sales_data_clean.xlsx",index=False,sheet_name="sales_data")

In [14]:
!pip install mysql-connector-python sqlalchemy pandas

Defaulting to user installation because normal site-packages is not writeable


In [22]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

user = "root"
password = "mysql2026"
host = "localhost"
port = 3306
db = "esmel"

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}",
    echo=False
)

In [23]:
engine.connect()

In [24]:
df.to_sql(
    name="ecommerce_sales_data",
    con=engine,
    if_exists="replace",
    index=False
)

3500